In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import collections
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#From Github; how to replace and expand contactions
import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [3]:
import requests
from bs4 import BeautifulSoup
import re


reviews = []
ratings = [] 

neg_reviews = []
pos_reviews = []

review_corpus = []


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


base_url = 'http://www.influenster.com/reviews/madewell'

while True:
    
    
    r = requests.get(base_url,headers=headers)

    soup = BeautifulSoup(r.text, 'html.parser')
    
    review_divs = soup.findAll('div', attrs={'itemprop': re.compile("^reviewBody")})

    for review in review_divs:
        if len(review.contents) > 0:
#             print(review.contents[0].strip().lower())
            try:
                review = review.contents[0].strip().lower()
                review_corpus.append([review])
                #expandContractions will put the dictionary made earlier to replace the contractions in the reviews
                #Make sure to to run the cList dict cell or else there will be an error
                expandContractions(review)
                #Cleaning the lemmas or words in reviews now will make it easier when we start predictive modeling
                words = word_tokenize(review)
                words = word_tokenize(review.replace('\n',' '))
                clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
                characters_to_remove = ["''",'``','...']
                clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
                english_stops = set(stopwords.words('english'))
                clean_words = [word for word in clean_words if word not in english_stops]
                wordnet_lemmatizer = WordNetLemmatizer()
                lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
                print(lemma_list)
                reviews.append(lemma_list)
            except:
                continue
        #Here we are using a simple control flow to recode the ratings for our model. If rating is greater than 3 positive, else negative   
    
    rating = soup.find_all('div', attrs={'itemprop': 'ratingValue' , 'content': re.compile("^[0-9]+$")})
    #the rating is actually an image, so we need to convert it into a string and then to an integer
    for rate in rating:
        if len(re.split('[<> ]',str(rate))) > 2:
            spl = re.split('[<> ]',str(rate))
#                 print(re.split('[=""]',spl[2]))
            int_rating = int(re.split('[=""]',str(rate))[2])
            print(int_rating)
            ratings.append(int_rating)
                
    
    next_page = soup.findAll('link', attrs={'rel': re.compile("^next")})
    print(next_page)
    if next_page is None or len(next_page) == 0:
        base_url = None
        break
    elif len(next_page) > 0:
        base_url = next_page[0].get('href')
    else:
        base_url = None
    
    if base_url is None:
        break
        
for int_rating, review in zip(ratings, reviews):        
    if int_rating == 1 or int_rating == 2 or int_rating == 3:
        rating = 'neg'
        ratings.append('neg')
        neg_reviews.append(review)
    elif int_rating == 4 or int_rating == 5:
        rating = 'pos'
        ratings.append('pos')
        pos_reviews.append(review)

my_df = pd.DataFrame(review_corpus)

my_df.to_csv('revised_madewell_reviews.csv', index=False, header=False)

['surprise', 'love', 'madewell', 'love', 'j.crew', 'sister', 'brand', 'afterall', 'madewell', 'classic', 'chic', 'bit', 'boho', 'madewell', 'expensive', 'quality', 'support', 'price', 'tag', 'one', 'favorite', 'item', 'madewell', 'tote', 'bag', 'seen', 'ever', 'fashion', 'blogger', "'s", 'arm', 'purchased', 'mine', 'back', '2006', 'still', 'carry', 'pretty', 'often', 'leather', 'held', 'nicely', 'little', 'character', "'s", 'well', 'loved', 'little', 'booty', 'year', 'still', 'wear', 'wo', "n't", 'regret', 'big', 'purchase', "'ll", 'last', 'forever']
['shop', 'basically', 'shirts/tops', 'madewell', 'really', 'like', 'style', 'however', 'think', 'product', 'overpriced', '70-', '90', 'top', 'lot', 'jean', 'good', 'quality', 'amazing', 'program', 'donate', 'one', 'pair', 'jean', 'get', '20', 'jean', 'madewell', 'would', 'recommend', 'shop', 'tops/dress', 'consider', 'lower', 'price', 'item']
['madewell', 'j.', 'crew', '’', 'cooler', 'older', 'sister', '’', 'rustic', 'meet', 'boho', 'chic'

['say', 'sale', 'worth', 'sign', 'membership', 'keep', 'eye', 'sale', 'section', 'get', 'really', 'good', 'deal', 'got', 'two', 'famous', 'wrap', 'top', '20', 'pair', '100', 'jean', '29', 'see', 'graphic', 't-shirt', 'plain', 't-shirt', 'get', 'size', 'big', 'get', 'anyway', 'shirt', 'beyond', 'comfortable', 'super', 'versatile']
['madewell', 'one', 'favorite', 'store', 'love', '’', 'style', '’', 'perfect', 'blend', 'casual', 'formal', 'although', 'would', 'call', 'madewell', 'affordable', 'must', 'say', 'clothing', 'perfect', 'long', 'term', 'usage', 'buying', 'jean', 'madewell', 'great', 'investment']
['love', 'madewell', 'price', 'worth', 'item', 'get', 'quality', 'department', 'store', 'zara', 'without', 'spend', 'much', 'madewell', 'definitely', 'make', 'amazing', 'shoe', 'worth', 'price', 'gray', 'boot', 'still', 'wear']
['madewell', 'favorite', 'store', 'moment', 'quality', 'item', 'always', 'great', 'especially', 'jean', 'curvy', 'girl', 'like', '’', 'difficult', 'find', 'pair'

['love', 'madewell', 'item', 'bit', 'pricey', 'opinion', 'cute', 'trendy', 'fit', 'well', 'quite', 'piece', 'madewell', 'well', 'made', 'last', 'multiple', 'season', 'highly', 'recommend']
['love', 'madewell', "'s", 'look', 'pretty', 'much', 'want', 'everything', 'store', 'given', 'time', 'thing', "'ve", 'purchased', 'lasted', 'really', 'long', 'time', '--', 'post', 'wallet', 'particular', 'still', 'beautiful', 'first', 'got', "n't", 'see', 'need', 'charge', 'much', 't-shirt', 'example', 'nice', 'cut', "n't", 'last', 'longer', 'shirt', 'think', 'price', 'could', 'far', 'reasonable']
['madewell', 'definitely', 'life', 'name', 'employee', 'always', 'knowledgeable', 'helped', 'trying', 'find', 'jean', 'clothes', 'fashionable', 'well', 'made', 'know', 'last', 'long', 'time']
['love', 'madewell', 'cutest', 'product', 'high', 'end', 'quality', 'shopping', 'experience', 'always', 'amazing', 'adorable', 'madewell', 'time', 'expensive', 'side', 'product', 'carry', 'worth', 'opinion']
['little',

['madewell', 'go', 'clothing', 'brand', 'find', 'clothing', 'generally', 'good', 'quality', 'clothes', '’', 'trendy', '’', 'dated', 'season', '’', 'also', 'bought', 'lot', 'shoe', '’', 'happy', 'quality', 'footwear', 'particularly', 'boot', 'great', 'sale', 'sale', 'promotion', 'every', 'often', 'also', 'always', 'offer', 'free', 'shipping']
['love', 'place', 'pricier', 'side', 'denim', 'amazing', 'quality', 'compared', 'many', 'retailer', 'definitely', 'recommend', 'nicer', 'durable', 'piece', 'want', 'wear', 'year', 'come']
["'ve", 'shopped', 'madewell', 'twice', 'pricing', 'expensive', 'hence', '4', 'star', 'instead', '5', 'however', 'every', 'item', 'ever', 'brought', 'madewell', 'far', 'ultimate', 'favorite', 'item', 'wardrobe', 'quality', 'really', 'like', 'wish', 'tad', 'le', 'expensive', 'get', 'pay']
['madewell', 'jean', 'absolute', 'ride', 'die', 'great', 'fit', 'slightly', 'curvier', 'side', '’', 'dip', 'back', 'large', 'backside', 'downside', 'pricing']
['boyfriend', 'backp

['t-shirt', 'place', 'great', 'allow', 'feel', 'comfortable', 'mom', 'bod', 'still', 'give', 'confidence', 'hip', 'style', 'high', 'budget', 'occasionally', 'splurge', 'run', 'great', 'deal']
['besides', 'sizing', 'love', 'everything', 'madewell', 'vintage', 'feel']
['great', 'brand', 'love', 'sweater', 'winter', 'time', 'store', 'always', 'clean', 'staff', 'friendly', 'always', 'new', 'item', 'great', 'accessory']
['madewell', 'quickly', 'become', 'go', 'store', 'past', '2', 'year', 'item', 'affordable', 'amazing', 'quality', 'purse', 'clothing', 'shoe', 'jewelry', 'amazing', 'love', 't-shirt', 'short', 'sleeve', 'button', 'item', 'great', 'staple', 'classic', 'easy', 'dress']
['recently', 'purchased', 'medium', 'transport', 'tote', 'love', '’', 'perfect', 'size', 'petite', 'girl', 'like', 'always', 'feel', 'like', 'larger', 'bag', 'may', 'drown', 'make', 'look', 'even', 'petite', 'medium', 'transport', 'favorite', 'bag', 'moment', 'simple', 'perfect', 'everyday', 'use']
['’', 'knocki

...

['great', 'quality', 'product', 'bit', 'overpriced', 'typically', 'purchase', 'sale', 'clearance', 'love', 'sweater', 'knit']
['go-to', 'store', 'buying', 'denim', 'love', 'fit', 'skinny', 'jean', 'wear', 'everything', 'great', 'quality', 'denim']
['item', 'live', 'name', 'especially', 'love', 'jean', 'denim', 'great', 'quality', 'fit', 'amazing']
['good', 'quality', '“', 'staple', '”', 'piece', 'last', 'forever', 'pair', 'many', 'different', 'outfits-', 'great', 'quality', 'versatility']
['love', 'madewell', 'find', 'thing', 'know', 'nobody', 'else', 'wearing', 'everything', 'great', 'quality', 'price']
['love', 'product', 'uniqueness', 'one', 'quality', 'fantastic', 'however', 'price', 'way', 'expensive']
['madewell', 'make', 'feel', 'professional', 'also', 'young', 'hot', 'price', "n't", 'crazy', 'catch', 'sale', 'good', 'time']
['probably', 'favorite', 'store', 'ever', 'wish', 'little', 'cheaper', '’', 'beat', 'quality', 'fun', 'style']
['like', 'boho-chic', 'style', 'well', 'answe

['madewell', 'lot', 'cute', 'trendy', 'clothing', 'love', 'store', 'much', 'always', 'find', 'something']
['love', 'madewell', 'clothes', 'theyre', 'bit', 'pricey', 'side', 'dont', 'shop', 'often', 'love']
['madewell', 'make', '50', 'wardobe', 'best', 'denim', 'great', 'place', 'go', 'fashionable', 'basic']
['item', 'wonderful', 'quality', 'timeless', 'fashion', 'piece', 'continue', 'shop', 'store']
['cutest', 'jewelry', 'gain', 'inspiration', 'style', 'love', 'bohemian', 'feel', 'stuff']
['bought', 'backpack', 'madewell', 'december', '’', 'favorite', 'bag', 'great', 'quality', 'great', 'customer', 'service']
['one', 'favorite', 'store', 'clothes', 'accessory', 'handbag', 'great', 'quality', 'leather', 'bag', 'every', 'occasion']
['nice', 'costumer', 'service', 'store', 'love', 'quality', 'thing', 'one', 'day', 'one', 'tote']
['love', 'brand', 'like', 'hipster', 'version', 'jcrew', 'perfect', 'casual/everyday', 'piece', 'great', 'quality']
['love', 'madewell', 'jean', 'really', 'comfor

['love', 'madewell', '’', 'definitely', 'bit', 'pricey', 'great', 'staple', 'splash', 'trendy']
['pricey', 'j', 'crew', "'s", 'little', 'sister', 'brand', 'shown', 'lot', 'improvement', 'past', 'year']
['favorite', 'store', 'ever', 'trendy', 'cute', 'always', 'find', 'something', 'want', 'great', 'quality']
['always', 'fit', 'like', 'glove', 'love', 'accessory', 'well', 'pricey', 'worth', 'quality']
['favorite', 'store', 'love', 'everything', 'clothes', 'shoe', 'jewelry', 'accessory']
['really', 'like', 'brand', 'several', 'color', 'v', 'neck', 'tshirts', 'wear', 'quite', 'often']
['clothes', 'young', 'hipster', "'ve", 'got', 'lot', 'nice', 'timeless', 'staple']
['great', 'quality', 'good', 'get', 'better', 'piece', 'keep', 'staple', 'item']
['madewell', 'definitely', 'favorite', 'store', 'shop', 'beautiful', 'casual', 'style', 'unique', 'twist']
['love', 'buying', 'jean', 'fit', 'well', 'good', 'price', 'would', 'highly', 'recommend', 'denim']
5
4
5
5
5
5
4
4
5
5
[<link href="https://

['recently', 'bought', 'jean', 'turned', 'well', 'fit', 'perfect', '’', 'good', 'quality']
['excellent', 'quality', 'clothing', 'accessory', 'always', 'changing', 'store', 'always', 'new', 'item', 'found', 'see', 'something', 'like', 'buy', 'may', 'store', 'following', 'week']
['madewell', 'great', 'quality', 'clothes', 'denim', 'amazing', 'best', 'basic', 'come', 'madewell', 'lot', 'stock', 'basic', 'always', 'nice', 'thick', 'fabric', 'definitely', 'minimalistic', 'look', 'cute']
['love', 'madewell', 'item', 'definitely', 'overpriced', 'however', 'jean', 'fit', 'great', 'made', 'great', 'material', 'highly', 'recommend', 'also', 'great', 'basic', 'cute', 'top', 'well', 'great', 'selection', 'bandana', 'personally', 'love', 'tote', 'bag', 'well']
['pricey', 'bag', 'shirt', 'lasted', 'long', 'time', 'good', 'quality']
['obsessed', 'madewell', 'make', 'cute', 'clothes', 'still', 'affordable', 'style', 'like', 'store']
['store', 'produce', 'high', 'quality', 'trendy', 'clothing', 'little

['absolutely', 'love', 'madewell', 'fit', 'style', 'perfectly', 'lot', 'laid', 'back', 'piece', 'also', 'smarter', 'piece', 'wear', 'everyday', 'work', 'nice', 'dinner', 'price', 'point', 'high', 'feel', 'like', 'quality', 'clothing', 'shoe', 'hand', 'bag', 'high', 'tend', 'shop', 'mostly', 'sale', 'get', 'great', 'item', 'good', 'price', 'running', 'lot', 'promotion', 'recently', '30-40', 'extra', 'sale', 'price', 'brings', '120', 'shirt', '59', 'bought', 'great', 'sweater', '140', 'originally', '59', 'wool', 'thick', "'s", 'super', 'warm', 'also', 'shop', 'memphis', 'store', 'mainly', 'love', 'selection', 'staff', 'also', 'friendly', 'found', 'item', 'liked', "n't", 'size', 'ordered', 'another', 'store', 'shipped', 'house', 'free', 'also', 'sale', 'item', 'arrived', 'two', 'business', 'day']
['madewell', 'one', 'favorite', 'brand', 'boyfriend', 'high', 'waisted', 'jean', 'beat', 'color', 'fade', 'one', 'criticism', 'expensive', 'really', 'go', 'treat']
['wish', 'canada', 'love', 'lea

In [17]:
print(len(neg_reviews))

101


In [18]:
data_words_pos = []

data_words_neg = []

for wd_list in pos_reviews:
    for word in wd_list:
        data_words_pos.append(word)
        
for wd_list in neg_reviews:
    for word in wd_list:
        data_words_neg.append(word)

In [19]:
data_words_pos = [data_words_pos]

data_words_neg = [data_words_neg]

In [20]:
# Build the bigram and trigram models
bigram_pos = gensim.models.Phrases(data_words_pos, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_pos = gensim.models.Phrases(bigram_pos[data_words_pos], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_pos = gensim.models.phrases.Phraser(bigram_pos)
trigram_mod_pos = gensim.models.phrases.Phraser(trigram_pos)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [21]:
bigram_neg = gensim.models.Phrases(data_words_neg, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_neg = gensim.models.Phrases(bigram_neg[data_words_neg], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_neg = gensim.models.phrases.Phraser(bigram_neg)
trigram_mod_neg = gensim.models.phrases.Phraser(trigram_neg)

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [22]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams_pos(texts):
    return [bigram_mod_pos[doc] for doc in texts]

def make_trigrams_pos(texts):
    return [trigram_mod_pos[bigram_mod_pos[doc]] for doc in texts]

def make_bigrams_neg(texts):
    return [bigram_mod_neg[doc] for doc in texts]

def make_trigrams_neg(texts):
    return [trigram_mod_neg[bigram_mod_neg[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [23]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [24]:
# Remove Stop Words
data_words_pos_nostops = remove_stopwords(data_words_pos)

data_words_neg_nostops = remove_stopwords(data_words_neg)

# Form Bigrams
data_words_bigrams_pos = make_bigrams_pos(data_words_pos_nostops)

data_words_bigrams_neg = make_bigrams_neg(data_words_neg_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv

nlp.max_length = 15000000

data_lemmatized_pos = lemmatization(data_words_bigrams_pos, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized_neg = lemmatization(data_words_bigrams_neg, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [25]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized_pos)

# Create Corpus
texts = data_lemmatized_pos

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

id2word_neg = corpora.Dictionary(data_lemmatized_neg)

# Create Corpus
texts = data_lemmatized_neg

# Term Document Frequency
corpus_neg = [id2word_neg.doc2bow(text) for text in texts]

# View
# print(corpus[:1])

In [26]:
import pprint

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model_neg = gensim.models.ldamodel.LdaModel(corpus=corpus_neg,
                                           id2word=id2word_neg,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\n\n\n')
pprint.pprint(lda_model_neg.print_topics())
doc_lda_neg = lda_model_neg[corpus]

[(0,
  '0.034*"love" + 0.033*"madewell" + 0.026*"quality" + 0.024*"jean" + '
  '0.021*"store" + 0.020*"great" + 0.015*"clothe" + 0.014*"good" + '
  '0.012*"always" + 0.011*"favorite"'),
 (1,
  '0.004*"love" + 0.003*"quality" + 0.003*"madewell" + 0.003*"store" + '
  '0.002*"jean" + 0.002*"great" + 0.002*"clothe" + 0.002*"make" + 0.002*"good" '
  '+ 0.002*"favorite"'),
 (2,
  '0.002*"love" + 0.002*"jean" + 0.002*"madewell" + 0.002*"quality" + '
  '0.002*"great" + 0.001*"always" + 0.001*"clothe" + 0.001*"store" + '
  '0.001*"favorite" + 0.001*"good"'),
 (3,
  '0.004*"madewell" + 0.003*"love" + 0.003*"jean" + 0.003*"great" + '
  '0.002*"store" + 0.002*"clothe" + 0.002*"quality" + 0.002*"always" + '
  '0.002*"good" + 0.002*"sale"'),
 (4,
  '0.003*"madewell" + 0.003*"jean" + 0.002*"love" + 0.002*"quality" + '
  '0.002*"great" + 0.002*"store" + 0.002*"clothe" + 0.001*"good" + '
  '0.001*"favorite" + 0.001*"always"'),
 (5,
  '0.006*"love" + 0.004*"madewell" + 0.003*"quality" + 0.003*"great" + 

In [27]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x             y  topics  cluster       Freq
topic                                                    
0     -0.221214 -7.108166e-08       1        1  99.999779
6      0.070547 -3.816373e-05       2        1   0.000057
2      0.041639  1.500835e-04       3        1   0.000033
7      0.035929  9.497672e-05       4        1   0.000030
4      0.030231 -2.013056e-04       5        1   0.000026
8      0.015750 -3.733232e-05       6        1   0.000017
1      0.015037 -4.351536e-05       7        1   0.000017
3      0.011443 -1.901507e-04       8        1   0.000015
5      0.008083  2.894910e-04       9        1   0.000013
9     -0.007445 -2.401259e-05      10        1   0.000009, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
1038  Default  780.000000        love  780.000000  30.0000  30.0000
1056  Default  769.000000    madewell  769.000000  29.0000  29.0000
1407  Default  610.000000     quality  610.000000  28.0000  28.0000
933   Default  554.000000        jean  554.000000  27.0000  27.0000
1697  Default  481.000000       store  481.000000  26.0000  26.0000
767   Default  468.000000       great  468.000000  25.0000  25.0000
303   Default  340.000000      clothe  340.000000  24.0000  24.0000
757   Default  329.000000        good  329.000000  23.0000  23.0000
56    Default  266.000000      always  266.000000  22.0000  22.0000
656   Default  260.000000    favorite  260.000000  21.0000  21.0000
1364  Default  252.000000       price  252.000000  20.0000  20.0000
1067  Default  254.000000        make  254.000000  19.0000  19.0000
1721  Default  234.000000       style  234.000000  18.0000  18.0000
1962  Default  238.000000        well  238.000000  17.0000  17.0000
1514  Default  221.000000        sale  221.000000  16.0000  16.0000
673   Default  223.000000         fit  223.000000  15.0000  15.0000
304   Default  218.000000    clothing  218.000000  14.0000  14.0000
1574  Default  210.000000        shop  210.000000  13.0000  13.0000
742   Default  204.000000         get  204.000000  12.0000  12.0000
200   Default  206.000000       brand  206.000000  11.0000  11.0000
409   Default  201.000000        cute  201.000000  10.0000  10.0000
1312  Default  193.000000       piece  193.000000   9.0000   9.0000
752   Default  194.000000          go  194.000000   8.0000   8.0000
228   Default  187.000000         buy  187.000000   7.0000   7.0000
929   Default  178.000000        item  178.000000   6.0000   6.0000
58    Default  171.000000     amazing  171.000000   5.0000   5.0000
616   Default  168.000000   expensive  168.000000   4.0000   4.0000
669   Default  166.000000        find  166.000000   3.0000   3.0000
592   Default  163.000000  everything  163.000000   2.0000   2.0000
446   Default  160.000000       denim  160.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
1407  Topic10    0.000009     quality  610.831787  -1.7281  -5.3602
1697  Topic10    0.000008       store  481.902008  -1.6392  -5.5083
767   Topic10    0.000006       great  468.838684  -1.8390  -5.7357
1056  Topic10    0.000007    madewell  769.935669  -2.2127  -5.6133
303   Topic10    0.000005      clothe  340.048370  -1.7345  -5.9523
1364  Topic10    0.000005       price  252.883148  -1.5646  -6.0786
1312  Topic10    0.000004       piece  193.353745  -1.4212  -6.2036
409   Topic10    0.000004        cute  201.377640  -1.4559  -6.1976
929   Topic10    0.000004        item  178.514053  -1.3968  -6.2590
742   Topic10    0.000004         get  204.249557  -1.4950  -6.2225
757   Topic10    0.000004        good  329.348419  -1.8445  -6.0943
1721  Topic10    0.000004       style  234.990356  -1.6261  -6.2135
1962  Topic10    0.000004        well  238.070938  -1.6529  -6.2272
200   Topic10    0.000004       brand  206.267883  -1.5807  -6.2984
1514  Topic10    0.000004        sale  221.143646  -1.6484  -6.29

In [28]:
vis_neg = pyLDAvis.gensim.prepare(lda_model_neg, corpus_neg, id2word_neg)
vis_neg

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0     -0.121948 -0.0       1        1  99.994652
4      0.019747 -0.0       2        1   0.000753
8      0.017532 -0.0       3        1   0.000676
9      0.016914  0.0       4        1   0.000657
2      0.015706  0.0       5        1   0.000628
5      0.014908  0.0       6        1   0.000607
1      0.011898  0.0       7        1   0.000553
6      0.010882 -0.0       8        1   0.000529
7      0.007288  0.0       9        1   0.000479
3      0.007074 -0.0      10        1   0.000476, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
291   Default  64.000000        love  64.000000  30.0000  30.0000
293   Default  59.000000    madewell  59.000000  29.0000  29.0000
386   Default  44.000000     quality  44.000000  28.0000  28.0000
262   Default  41.000000        jean  41.000000  27.0000  27.0000
468   Default  39.000000       store  39.000000  26.0000  26.0000
223   Default  35.000000       great  35.000000  25.0000  25.0000
221   Default  30.000000        good  30.000000  24.0000  24.0000
188   Default  29.000000    favorite  29.000000  23.0000  23.0000
17    Default  23.000000      always  23.000000  22.0000  22.0000
392   Default  22.000000      really  22.000000  21.0000  21.0000
296   Default  22.000000        make  22.000000  20.0000  20.0000
374   Default  20.000000       price  20.000000  19.0000  19.0000
196   Default  20.000000         fit  20.000000  18.0000  18.0000
215   Default  19.000000         get  19.000000  17.0000  17.0000
95    Default  19.000000      clothe  19.000000  16.0000  16.0000
68    Default  17.000000       brand  17.000000  15.0000  15.0000
73    Default  17.000000         buy  17.000000  14.0000  14.0000
548   Default  16.000000        well  16.000000  13.0000  13.0000
480   Default  16.000000       style  16.000000  12.0000  12.0000
259   Default  15.000000        item  15.000000  11.0000  11.0000
96    Default  15.000000    clothing  15.000000  10.0000  10.0000
288   Default  14.000000        look  14.000000   9.0000   9.0000
285   Default  14.000000      little  14.000000   8.0000   8.0000
219   Default  14.000000          go  14.000000   7.0000   7.0000
236   Default  14.000000        high  14.000000   6.0000   6.0000
169   Default  14.000000  everything  14.000000   5.0000   5.0000
129   Default  13.000000        cute  13.000000   4.0000   4.0000
509   Default  13.000000        time  13.000000   3.0000   3.0000
414   Default  13.000000        sale  13.000000   2.0000   2.0000
351   Default  13.000000        pair  13.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
221   Topic10   0.000023        good  30.186031  -1.8323  -5.9666
223   Topic10   0.000023       great  35.046097  -1.9735  -5.9585
262   Topic10   0.000023        jean  41.853317  -2.1648  -5.9723
548   Topic10   0.000021        well  16.596378  -1.3399  -6.0724
468   Topic10   0.000022       store  39.916367  -2.1363  -5.9912
374   Topic10   0.000021       price  20.479393  -1.5493  -6.0715
17    Topic10   0.000021      always  23.398226  -1.6877  -6.0767
188   Topic10   0.000021    favorite  29.244507  -1.9032  -6.0692
95    Topic10   0.000020      clothe  19.522087  -1.5502  -6.1203
296   Topic10   0.000020        make  22.439482  -1.6883  -6.1191
392   Topic10   0.000020      really  22.426962  -1.6931  -6.1245
480   Topic10   0.000019       style  16.606604  -1.4171  -6.1490
196   Topic10   0.000019         fit  20.493700  -1.6172  -6.1388
259   Topic10   0.000019        item  15.630731  -1.3780  -6.1704
68    Topic10   0.000019       brand  17.570293  -1.4895  -6.1650
288   Topic10   0.000018        look  14.660644  -1.3303  -6.1869
452   Topic10   0.000018   something   9.801830  -0.9527  -6.2119
236   Topic10   0.000018        high  14.665620  -1.3348  -6.1910
129

[  Top Competitors  Websites Market Share             Versus Page
 0          Stripe      3793       72.29%       PayPal vs. Stripe
 1          Klarna       303        5.77%       PayPal vs. Klarna
 2         GiroPay        42         0.8%      PayPal vs. GiroPay
 3     PaySafeCard        34        0.65%  PayPal vs. PaySafeCard
 4         Paysafe        15        0.29%      PayPal vs. Paysafe]